# NHL 2009-2018 Draft dataset

# NHL API

In [ ]:
import requests

## NHL Stats API endpoints

* Divisions
* Conferences
* Game
* Tournaments (playoffs)
* Schedule
* Seasons
* Standings
* Awards
* Venues
* Teams  

GET https://statsapi.web.nhl.com/api/v1/teams   
Returns a list of data about all teams including their id, venue details, division, conference and franchise information.  

GET https://statsapi.web.nhl.com/api/v1/teams/ID/roster  
Returns entire roster for a team including id value, name, jersey number and position details.

* People

**GET** https://statsapi.web.nhl.com/api/v1/people/ID  
Gets details for a player, must specify the id value in order to return data.

**GET** https://statsapi.web.nhl.com/api/v1/people/ID/stats  
Complex endpoint with lots of append options to change what kind of stats you wish to obtain

_Modifiers_  
?stats=statsSingleSeason&season=19801981   
Obtains single season statistics for a player  

?stats=homeAndAway&season=20162017  
Provides a split between home and away games.

?stats=winLoss&season=20162017  
Very similar to the previous modifier except it provides the W/L/OT split instead of Home and Away

?stats=byMonth&season=20162017  
Monthly split of stats

?stats=byDayOfWeek&season=20162017  
Split done by day of the week

?stats=vsDivision&season=20162017  
Division stats split

?stats=vsConference&season=20162017  
Conference stats split

?stats=vsTeam&season=20162017  
Conference stats split

?stats=gameLog&season=20162017  
Provides a game log showing stats for each game of a season

?stats=regularSeasonStatRankings&season=20162017   
Returns where someone stands vs the rest of the league for a specific regularSeasonStatRankings

?stats=goalsByGameSituation&season=20162017  
Shows number on when goals for a
player happened like how many in the shootout, how many in each period, etc.

?stats=onPaceRegularSeason&season=20172018  
This only works with the current in-progress season and shows projected totals based on current onPaceRegularSeason

* Draft

GET https://statsapi.web.nhl.com/api/v1/draft  
Get round-by-round data for current year's NHL Entry Draft.

**GET** https://statsapi.web.nhl.com/api/v1/draft/YEAR  
Takes a YYYY format year and returns draft data

* Prospects

**GET** https://statsapi.web.nhl.com/api/v1/draft/prospects  
Get all NHL Entry Draft prospects.

GET https://statsapi.web.nhl.com/api/v1/draft/prospects/ID  
Get an NHL Entry Draft prospect.

In [ ]:
url = ''